# Community Question Answering (Subtask B)

In [1]:
from __future__ import print_function

from keras.preprocessing.text import *
from keras.preprocessing.sequence import *
from keras.models import *
from keras.layers import *
from keras.utils import *
from keras import backend as K
from keras.engine.topology import Layer
from keras.callbacks import ModelCheckpoint
import time
import numpy as np
import json
import h5py
from google.colab import files

Using TensorFlow backend.


In [2]:
import tensorflow, keras
print("Tensorflow version ", tensorflow.__version__)
print("Keras version ", keras.__version__)

from tensorflow.python.client import device_lib
device_lib.list_local_devices()


Tensorflow version  1.8.0
Keras version  2.1.6


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2029353060459652342, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11287966516
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 12570430546364783582
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

##acquire dataset

In [3]:
# download and unzip embeddings and dataset
!wget -O glove.zip http://nlp.stanford.edu/data/glove.6B.zip
!unzip -o glove.zip -d glove
!wget -O task3-cqa.zip http://alt.qcri.org/semeval2016/task3/data/uploads/semeval2016-task3-cqa-ql-traindev-v3.2.zip
!unzip -o task3-cqa.zip -d data
!wget -O test.zip http://alt.qcri.org/semeval2017/task3/data/uploads/semeval2017_task3_test.zip
#!wget -O test.zip http://alt.qcri.org/semeval2017/task3/data/uploads/semeval2017_task3_test_input_abcd.zip
!unzip -o test.zip -d test_data

--2018-06-11 18:39:18--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2018-06-11 18:39:19--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.zip’



glove.zip           100%[===================>] 822.24M  14.7MB/s    in 43s     

2018-06-11 18:40:02 (19.0 MB/s) - ‘glove.zip’ saved [862182613/862182613]

Archive:  glove.zip
  inflating: glove/glove.6B.50d.txt  
  inflating: glove/glove.6B.100d.txt  
  inflating: glove/glove.6B.200d.txt  
  inflating: glove/glove.6B.300d.txt  
--2018-06-11 18:40:27--  http://alt.qcri.org/semeval2016/task3/data/uploads/semeval2016-task3-cqa-ql-traindev-v3.2.zip
Resolving alt.qcri.org (alt.qcri.org)... 212.71.235.101
Connecting to alt.qcri.org (alt.qcri.org)|212.71.235.101|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21555267 (21M) [application/zip]
Saving to: ‘task3-cqa.zip’

task3-cqa.zip       100%[===================>]  20.56M  14.4MB/s    in 1.4s    

2018-06-11 18:40:29 (14.4 MB/s) - ‘task3-cqa.zip’ saved [21555267/21555267]

Archive:  task3-cqa.zip
   creating: data/v3.2/
  inflating: data/v3.2/.DS_Store     
   creating: data/__MACOSX/
   creating: data/__MACOSX/v3.2

test.zip            100%[===================>]   6.28M  7.73MB/s    in 0.8s    

2018-06-11 18:40:33 (7.73 MB/s) - ‘test.zip’ saved [6586572/6586572]

Archive:  test.zip
   creating: test_data/SemEval2017_task3_test/
  inflating: test_data/SemEval2017_task3_test/.DS_Store  
   creating: test_data/__MACOSX/
   creating: test_data/__MACOSX/SemEval2017_task3_test/
  inflating: test_data/__MACOSX/SemEval2017_task3_test/._.DS_Store  
   creating: test_data/SemEval2017_task3_test/Arabic/
  inflating: test_data/SemEval2017_task3_test/Arabic/.DS_Store  
   creating: test_data/__MACOSX/SemEval2017_task3_test/Arabic/
  inflating: test_data/__MACOSX/SemEval2017_task3_test/Arabic/._.DS_Store  
  inflating: test_data/SemEval2017_task3_test/Arabic/SemEval2017-Task3-CQA-MD-test.xml.gz  
  inflating: test_data/__MACOSX/SemEval2017_task3_test/Arabic/._SemEval2017-Task3-CQA-MD-test.xml.gz  
  inflating: test_data/__MACOSX/SemEval2017_task3_test/._Arabic  
   creating: test_data/SemEval2017_task3_test/En

In [0]:
train_dir = 'data/v3.2/train/'
dev_dir = 'data/v3.2/dev/'
test_dir = 'test_data/SemEval2017_task3_test/English/'

##preprocessing classes

In [0]:
class MapHelper:
    
    def append_to_list(map_with_list_as_value, map_key, new_list_element):
        existing_list = map_with_list_as_value.get(map_key, [])
        new_list = existing_list + [new_list_element]
        map_with_list_as_value[map_key] = new_list
    
    def append_to_set(map_with_set_as_value, map_key, new_set_element):
        existing_set = map_with_set_as_value.get(map_key, set())
        existing_set.add(new_set_element)
        map_with_set_as_value[map_key] = existing_set

In [0]:
class QuestionRelevance:
    """
        Makes no distinction between 
        PerfectMatch and Relevant questions
    """
    
    INT_RELEVANT = 1
    RELEVANT = "Relevant"
    
    PERFECT_MACH = "PerfectMatch"
    
    INT_IRRELEVANT = 0
    IRRELEVANT = "Irrelevant"
    
    
    def to_number(string_relevance):
        return QuestionRelevance.INT_IRRELEVANT\
            if string_relevance == QuestionRelevance.IRRELEVANT \
            else QuestionRelevance.INT_RELEVANT
    
    
    def from_number(int_relevance):
        return QuestionRelevance.IRRIELEVANT \
            if string_relevance == QuestionRelevance.INT_IRRELEVANT \
            else QuestionRelevance.RELEVANT

In [0]:
class Thread:
    
    def __init__(self, question, rel_questions):
        self.question = question
        self.rel_questions = rel_questions

    def __str__(self):
        return self.question.__str__()
    
    def __repr__(self):
        return self.question.__repr__()

In [0]:
class ThreadCollection:
    
    def __init__(self):
        self.by_topic = {}
    
    def add(self, string_topic, thread):
        MapHelper.append_to_list(self.by_topic, string_topic, thread)
    
    def get(self, string_topic):
        return self.by_topic.get(string_topic, [])
    
    def topics(self):
        return list(self.by_topic.keys())

    def topic_thread_pairs(self):
        return self.by_topic.items()

In [0]:
from collections import namedtuple


class QuestionData:
  
    STRING_ID_KEY = "ORGQ_ID"
    
    def __init__(self, id, subject, body):
        self.id = id
        self.subject = subject
        self.body = body
    
    def __str__(self):
        return str(self.body)
    
    def __repr__(self):
        return self.__str__()
        
        
class RelQuestionData:
    
    STRING_RELEVANCE_KEY = "RELQ_RELEVANCE2ORGQ"
    STRING_USERNAME_KEY = "RELQ_USERNAME"
    STRING_DATE_KEY = "RELQ_DATE"
    STRING_ID_KEY = "RELQ_ID"
    
    def __init__(self, metadata, subject, body):
        self.metadata = metadata
        self.subject = subject
        self.body = body
        
    def __str__(self):
        return str(self.body)
    
    def __repr__(self):
        return self.__str__()

    
class Metadata:
    
    def __init__(self, id, user, date, relevance):
        self.id = id
        self.user = user
        self.date = date
        self.relevance = relevance
    
    
QuestionQuestionPair = namedtuple('QuestionQuestionPair', ['question', 'rel_question', 'data'])

# set data to None by default
QuestionQuestionPair.__new__.__defaults__ = (None,)

In [0]:
def to_question_data(node_question):
    subject = node_question[0]
    body = node_question[1]
    
    return QuestionData(node_question.attrib[QuestionData.STRING_ID_KEY], subject.text, body.text)


def to_rel_question_data(node_rel_questions):
    f = lambda node: RelQuestionData(Metadata(node.attrib[RelQuestionData.STRING_ID_KEY],
                                         node.attrib[RelQuestionData.STRING_USERNAME_KEY],
                                         node.attrib[RelQuestionData.STRING_DATE_KEY],
                                         QuestionRelevance.to_number(node.attrib[RelQuestionData.STRING_RELEVANCE_KEY])), 
                                node[0].text, node[1].text)
    
    return [f(node) for node in node_rel_questions if node[1].text is not None]

In [0]:
from nltk.corpus.reader import XMLCorpusReader, XMLCorpusView

def load_data(string_basedir, string_filenames_in_basedir):
    
    for string_filename in string_filenames_in_basedir:
        corpus_reader = XMLCorpusReader(string_basedir, string_filename)
        i = 0
        node_rel_questions = []
        for root_node in corpus_reader.xml():
            node_question = root_node
            node_rel_questions.append(root_node[2][0])
            i += 1
            if i == 10:
                thread_question = to_question_data(node_question)
                thread_rel_questions = to_rel_question_data(node_rel_questions)

                thread = Thread(thread_question, thread_rel_questions)        
                string_topic = ""
                
                i = 0
                node_rel_questions = []
            
                yield string_topic, thread

In [0]:
from itertools import *


def get_limited_generator(string_basedir,
                          list_string_filenames_in_basedir,
                          start_index_inclusive, 
                          end_index_exclusive = None, 
                          step = None):
    data_generator = load_data(string_basedir, list_string_filenames_in_basedir)
    return islice(data_generator, start_index_inclusive, end_index_exclusive, step)

In [0]:
def repeat_relevant(question, list_relevant, int_how_many):
    for i in range(int_how_many):
        question = QuestionData(str(question.subject), str(question.body))
        yield QuestionQuestionPair(question, list_relevant[i % len(list_relevant)] )
    

def get_data(data_generator, do_repeat_relevant = False):
    X = []
    y = []
    
    for topic, thread in data_generator:
        question = thread.question
        list_relevant = []
        
        for rel_question in thread.rel_questions:
            question = QuestionData(str(question.id), str(question.subject), str(question.body))
            question_question_pair = QuestionQuestionPair(question, rel_question)
            
            X += [ question_question_pair ]
            y += [ rel_question.metadata.relevance ]
    
            if rel_question.metadata.relevance == QuestionRelevance.INT_RELEVANT:
                list_relevant += [rel_question]
        
        num_relevant = len(list_relevant)
        num_irrelevant = len(thread.rel_questions) - num_relevant
        
        if num_relevant < num_irrelevant \
            and do_repeat_relevant \
            and num_relevant != 0:
                
            delta = num_irrelevant - num_relevant
            
            X += repeat_relevant(question, list_relevant, delta)
            y += [ QuestionRelevance.INT_RELEVANT for i in range(delta) ]
        
    return X, y

##dataset loading

In [0]:
train_generator = get_limited_generator(train_dir, ['SemEval2016-Task3-CQA-QL-train-part1.xml', 'SemEval2016-Task3-CQA-QL-train-part2.xml'], 
                                       0, None, None)

dev_generator = get_limited_generator(dev_dir, ['SemEval2016-Task3-CQA-QL-dev.xml'], 0, None, None)
test_generator = get_limited_generator(test_dir, ['SemEval2017-task3-English-test.xml'], 0, None, None)
#test_generator_A = get_limited_generator(test_dir, ['SemEval2017-task3-English-test-subtaskA.xml'], 0, None, None, subtask_A=True)

X = {"train": [], "dev":[], "test":[], "test_A":[]}
y = {"train": [], "dev":[], "test":[], "test_A":[]}

X["train"], y["train"] = get_data(train_generator, do_repeat_relevant=False)
X["dev"], y["dev"] = get_data(dev_generator, do_repeat_relevant=False)
X["test"], y["test"] = get_data(test_generator, do_repeat_relevant=False)
#X["test_A"], y["test_A"] = get_data(test_generator_A, do_repeat_relevant=False)

In [15]:
key = "test"
print(len(X[key]))
print(len(y[key]))
for i in range(5):
  x = X[key][i]
  print(x[0].id + " " + x[0].subject + " " + x[0].body)
  print(x[1].metadata.id + " " + x[1].subject + " " + x[1].body)
#print(X[0][0])
print(type(y[key]))
print(y[key][:5])

880
880
Q388 HOW TO APPLY AND OBTAIN VISAS TO UK AND SPAIN Can someone suggest a link or explain how to obtain visas for family hoping to go on vacation to The UK and Spain? We are bona fide residents in Qatar by the way. Thank you
Q388_R14 Schengen Visa @ Greece Embassy Do you know how long it will take to get Schengen Visa from Embassy of Greece? 3 days? 1 week? any idea? Thanks.
Q388 HOW TO APPLY AND OBTAIN VISAS TO UK AND SPAIN Can someone suggest a link or explain how to obtain visas for family hoping to go on vacation to The UK and Spain? We are bona fide residents in Qatar by the way. Thank you
Q388_R21 HELLLLPPPP! Now that I have your attention (good morning btw ;)... Does anyone know if transit visa's are issued here?
Q388 HOW TO APPLY AND OBTAIN VISAS TO UK AND SPAIN Can someone suggest a link or explain how to obtain visas for family hoping to go on vacation to The UK and Spain? We are bona fide residents in Qatar by the way. Thank you
Q388_R23 Swiss embassy Hi there. Can an

In [16]:
questions = {"train": [], "dev":[], "test":[], "test_A":[]}
answers = {"train": [], "dev":[], "test":[], "test_A":[]}
for key in X.keys():
  for x in X[key]:
    questions[key].append(x[0].body)
    answers[key].append(x[1].body)
  
#print(questions[:5])
#print(answers[:5])

tokenizer = Tokenizer(lower=True)
tokenizer.fit_on_texts(questions["train"] + answers["train"])
vocab_size = len(tokenizer.word_index)
print('Found %s unique tokens.' % vocab_size)

quest = {"train": [], "dev":[], "test":[], "test_A":[]}
answ = {"train": [], "dev":[], "test":[], "test_A":[]}

# integer encode the sentences
for key in questions.keys():
  quest[key] = tokenizer.texts_to_sequences(questions[key])
  
for key in answers.keys():
  answ[key] = tokenizer.texts_to_sequences(answers[key])

# prepare data for Keras network
MAX_SEQUENCE_LENGTH = 100

train_X = [pad_sequences(quest["train"], maxlen=MAX_SEQUENCE_LENGTH), pad_sequences(answ["train"], maxlen=MAX_SEQUENCE_LENGTH)]
dev_X = [pad_sequences(quest["dev"], maxlen=MAX_SEQUENCE_LENGTH), pad_sequences(answ["dev"], maxlen=MAX_SEQUENCE_LENGTH)]
test_X = [pad_sequences(quest["test"], maxlen=MAX_SEQUENCE_LENGTH), pad_sequences(answ["test"], maxlen=MAX_SEQUENCE_LENGTH)]
#test_A_X = [pad_sequences(quest["test_A"], maxlen=MAX_SEQUENCE_LENGTH), pad_sequences(answ["test_A"], maxlen=MAX_SEQUENCE_LENGTH)]

train_y = np.array(y["train"])
dev_y = np.array(y["dev"])
test_y = np.array(y["test"])
#test_A_y = np.array(y["test_A"])


Found 7980 unique tokens.


In [17]:
print(test_X[0][:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0   11  108  168
     5 1104   21  811   35    2 1235  585    8   67  763    2   68   26
   801    2    3  214    6 4488   46   18 1345    4   15   88    3  252
    87   14]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0   11  108  168
     5 1104   21  811   35    2 1235  585    8   67  763    2   68   26
   801    2    3  214    6 4488   46   18 1345    4

In [18]:
maxlen = 100

train_lens = [np.count_nonzero(t) for t in answ["train"] + quest["train"]]
x = len(train_lens)
sx = len([l for l in train_lens if l > maxlen])
print("Train sentences longer than {} tokens: {:.2f} %".format(maxlen,sx/x*100))

Train sentences longer than 100 tokens: 0.00 %


In [19]:
print(train_X[0][:5])
print(train_X[1][:5])
print(train_y[:5])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0  53   1  11 110  63 733   8 856]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0  53   1  11 110  63 733   8 856]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0 

##gloVe

In [20]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('glove/glove.6B.300d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size+1, 300))
for word, i in tokenizer.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector
    
print('Embedding matrix shape=', embedding_matrix.shape)

Loaded 400000 word vectors.
Embedding matrix shape= (7981, 300)


In [21]:
device_name = tensorflow.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

def myLSTM(*args, **kwargs):
  if device_name == '/device:GPU:0':
    return CuDNNLSTM(*args, **kwargs)
  return LSTM(*args, **kwargs)

def myGRU(*args, **kwargs):
  if device_name == '/device:GPU:0':
    return CuDNNGRU(*args, **kwargs)
  return GRU(*args, **kwargs)

Found GPU at: /device:GPU:0


In [0]:
!mkdir checkpoints

##evaluation

In [0]:
def predict(model, inputs, batch_size):
    probs = model.predict(inputs, batch_size=batch_size)
    return np.squeeze(probs), np.array(["false" if x < 0.5 else "true" for x in probs])

In [0]:
!mkdir preds

In [0]:
def create_pred_file(probs, preds, X, pred_filepath):
    with open(pred_filepath, 'w') as pred_file:
        for i in range(len(X)):
            x = X[i]
            print("{}\t{}\t{}\t{}\t{}".format(x[0].id, x[1].metadata.id, 0, probs[i], preds[i]), 
                  file=pred_file)
    files.download(pred_filepath)
    

def predict_and_save(model, processed_input, unprocessed_input, pred_filepath):
    probs, preds = predict(model, processed_input, BATCH_SIZE)
    create_pred_file(probs, preds, unprocessed_input, pred_filepath)

##final CNN model

In [0]:
#
# CNN classifer
#
from keras.optimizers import *
sgd = SGD(lr=0.1, momentum=0.1, decay=0.99, nesterov=True)
#adagrad = Adagrad(lr=0.5, epsilon=None, decay=0.09)
activation = 'elu'
padding = 'valid'
filters = 300
kernel_size = 3

HIDDEN = 300

print('Build model...')
inputs_q = Input(shape=(MAX_SEQUENCE_LENGTH,), name='question')
inputs_a = Input(shape=(MAX_SEQUENCE_LENGTH,), name='answer')

# add masking
mask = Masking(mask_value=0., name='masking')
inputs_q_ = mask(inputs_q)
inputs_a_ = mask(inputs_a)

embed = Embedding(vocab_size+1, HIDDEN, weights=[embedding_matrix], trainable=False, mask_zero=False, name='embedding')#mask_zero=True,trainable=False
embed_q = embed(inputs_q_)
embed_a = embed(inputs_a_)

# convolution one
conv1 = Conv1D(filters, kernel_size, name='conv1', padding='valid', activation=activation, strides=1)
conv1_q = conv1(embed_q)
conv1_a = conv1(embed_a)

pool1 = GlobalMaxPooling1D(name='pool1') #MAX
pool1_q = pool1(conv1_q)
pool1_a = pool1(conv1_a)

# convolution two
conv2 = Conv1D(filters, kernel_size, name='conv2', padding='valid', activation=activation, strides=1)
conv2_q = conv2(conv1_q)
conv2_a = conv2(conv1_a)

pool2 = GlobalMaxPooling1D(name='pool2') #MAX
pool2_q = pool2(conv2_q)
pool2_a = pool2(conv2_a)

# convolution three
conv3 = Conv1D(filters, kernel_size, name='conv3', padding='valid', activation=activation, strides=1)
conv3_q = conv1(conv2_q)
conv3_a = conv1(conv2_a)

pool3 = GlobalMaxPooling1D(name='pool3')
pool3_q = pool1(conv3_q)
pool3_a = pool1(conv3_a)

# convolution four
conv4 = Conv1D(filters, kernel_size, name='conv4', padding='valid', activation=activation, strides=1)
conv4_q = conv1(conv3_q)
conv4_a = conv1(conv3_a)

pool4 = GlobalMaxPooling1D(name='pool4')
pool4_q = pool1(conv4_q)
pool4_a = pool1(conv4_a)

# concatenations
q_ = concatenate([pool1_q, pool2_q], name='question_concat') #, pool3_q, pool4_q
a_ = concatenate([pool1_a, pool2_a], name='answer_concat') #, pool3_a, pool4_a
#re_q = Reshape((-1, HIDDEN))(q_)
#re_a = Reshape((-1, HIDDEN))(a_)
q = Dropout(rate=0.25, name='question_dropout')(q_)
a = Dropout(rate=0.25, name='answer_dropout')(a_)

#outs = Lambda(euclidean_similarity, name='euclid_similarity')([q,a])
#outs = Dot(axes=-1, normalize=True)([q,a])

qa_mul = multiply([q, a], name='multiply') 
qa_sub = subtract([q, a], name='subtract')
qa_max = maximum([q, a], name='maximum')
qa_avg = average([q, a], name='average')
concat = concatenate([q, a, qa_max, qa_mul, qa_sub, qa_avg], name='final_concat')

# attention block - multi-dimensional self-attention
reshaped = Reshape((-1, HIDDEN))(concat)
compatibilities = Dense(HIDDEN, activation='elu', use_bias=False, name='compatibilites')(reshaped)
attention_weights = Dense(HIDDEN, activation='softmax', use_bias=False, name='self_att_weights')(compatibilities)
attention_ = multiply([reshaped, attention_weights], name='attention_mul')
attention = Lambda(lambda x: K.sum(x, axis=1), name="attention")(attention_)
#attention = dot([reshaped, attention_weights], axes=-1, name='attention')

hidden_ = Dense(HIDDEN, activation='tanh', name='final_dense_1')(attention)
#hidden_2 = Dense(HIDDEN, activation='elu', name='final_dense_2')(hidden_1)
#hidden_ = Dropout(rate=0.25, name="hidden_dropout")(hidden_2)
#hidden = BatchNormalization()(hidden_)

preds = Dense(1, activation='sigmoid', name='predictions')(attention)#(hidden_)

CNN = Model(inputs=[inputs_q, inputs_a], outputs=[preds])
# try using different optimizers and different optimizer configs
CNN.compile(loss='binary_crossentropy',
              optimizer='adam',#'adadelta',
              metrics=['accuracy']) # add mean_average_precision ?
print(CNN.summary())

Build model...
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
question (InputLayer)           (None, 100)          0                                            
__________________________________________________________________________________________________
answer (InputLayer)             (None, 100)          0                                            
__________________________________________________________________________________________________
masking (Masking)               (None, 100)          0           question[0][0]                   
                                                                 answer[0][0]                     
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 300)     2394300     masking[0][0]                

In [0]:
BATCH_SIZE = 32 # set to max allowable value
EPOCHS = 10*3#10*2

print('Train...')
CNN.fit(train_X, train_y,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_data=(dev_X, dev_y),
          verbose=2,
          callbacks=None)

score, acc = CNN.evaluate(test_X, test_y, batch_size=BATCH_SIZE)
print('Test score:', score)
print('Test accuracy:', acc)
# save model
CNN.save('cnn_maxpool_final_B.h5')

Train...
Train on 2593 samples, validate on 495 samples
Epoch 1/30
 - 8s - loss: 0.6767 - acc: 0.5927 - val_loss: 0.6864 - val_acc: 0.5717
Epoch 2/30
 - 2s - loss: 0.6624 - acc: 0.5981 - val_loss: 0.6896 - val_acc: 0.5717
Epoch 3/30
 - 2s - loss: 0.6626 - acc: 0.5989 - val_loss: 0.6795 - val_acc: 0.5737
Epoch 4/30
 - 2s - loss: 0.6385 - acc: 0.6332 - val_loss: 0.6882 - val_acc: 0.5778
Epoch 5/30
 - 2s - loss: 0.6226 - acc: 0.6475 - val_loss: 0.6803 - val_acc: 0.5798
Epoch 6/30
 - 2s - loss: 0.5950 - acc: 0.6653 - val_loss: 0.7260 - val_acc: 0.5657
Epoch 7/30
 - 2s - loss: 0.5764 - acc: 0.7015 - val_loss: 0.7196 - val_acc: 0.6081
Epoch 8/30
 - 2s - loss: 0.5612 - acc: 0.7162 - val_loss: 0.7384 - val_acc: 0.5960
Epoch 9/30
 - 2s - loss: 0.5500 - acc: 0.7189 - val_loss: 0.7456 - val_acc: 0.5859
Epoch 10/30
 - 2s - loss: 0.5188 - acc: 0.7501 - val_loss: 0.7586 - val_acc: 0.5960
Epoch 11/30
 - 2s - loss: 0.5233 - acc: 0.7366 - val_loss: 0.8153 - val_acc: 0.5636
Epoch 12/30
 - 2s - loss: 0.5

In [0]:
predict_and_save(CNN, test_X, X["test"], "preds/cnn_maxpool_final_B.pred")

In [0]:
files.download('cnn_maxpool_final_B.h5')

In [0]:
#
# CNN classifer average pooling
#
from keras.optimizers import *
sgd = SGD(lr=0.1, momentum=0.1, decay=0.99, nesterov=True)
#adagrad = Adagrad(lr=0.5, epsilon=None, decay=0.09)
activation = 'elu'
padding = 'valid'
filters = 300
kernel_size = 3

HIDDEN = 300

print('Build model...')
inputs_q = Input(shape=(MAX_SEQUENCE_LENGTH,), name='question')
inputs_a = Input(shape=(MAX_SEQUENCE_LENGTH,), name='answer')

# add masking
mask = Masking(mask_value=0., name='masking')
inputs_q_ = mask(inputs_q)
inputs_a_ = mask(inputs_a)

embed = Embedding(vocab_size+1, HIDDEN, weights=[embedding_matrix], trainable=False, mask_zero=False, name='embedding')#mask_zero=True,trainable=False
embed_q = embed(inputs_q_)
embed_a = embed(inputs_a_)

# convolution one
conv1 = Conv1D(filters, kernel_size, name='conv1', padding='valid', activation=activation, strides=1)
conv1_q = conv1(embed_q)
conv1_a = conv1(embed_a)

pool1 = GlobalAveragePooling1D(name='pool1')
pool1_q = pool1(conv1_q)
pool1_a = pool1(conv1_a)

# convolution two
conv2 = Conv1D(filters, kernel_size, name='conv2', padding='valid', activation=activation, strides=1)
conv2_q = conv2(conv1_q)
conv2_a = conv2(conv1_a)

pool2 = GlobalAveragePooling1D(name='pool2')
pool2_q = pool2(conv2_q)
pool2_a = pool2(conv2_a)

# convolution three
conv3 = Conv1D(filters, kernel_size, name='conv3', padding='valid', activation=activation, strides=1)
conv3_q = conv1(conv2_q)
conv3_a = conv1(conv2_a)

pool3 = GlobalAveragePooling1D(name='pool3')
pool3_q = pool1(conv3_q)
pool3_a = pool1(conv3_a)

# convolution four
conv4 = Conv1D(filters, kernel_size, name='conv4', padding='valid', activation=activation, strides=1)
conv4_q = conv1(conv3_q)
conv4_a = conv1(conv3_a)

pool4 = GlobalAveragePooling1D(name='pool4')
pool4_q = pool1(conv4_q)
pool4_a = pool1(conv4_a)

# concatenations
q_ = concatenate([pool1_q, pool2_q, pool3_q, pool4_q], name='question_concat')
a_ = concatenate([pool1_a, pool2_a, pool3_a, pool4_a], name='answer_concat')
re_q = Reshape((-1, HIDDEN))(q_)
re_a = Reshape((-1, HIDDEN))(a_)
q = Dropout(rate=0.25, name='question_dropout')(re_q)#(q_)
a = Dropout(rate=0.25, name='answer_dropout')(re_a)#(a_)

qa_mul = multiply([q, a], name='multiply') 
qa_sub = subtract([q, a], name='subtract')
qa_max = maximum([q, a], name='maximum')
qa_avg = average([q, a], name='average')
concat = concatenate([q, a, qa_max, qa_mul, qa_sub, qa_avg], name='final_concat')

# attention block - multi-dimensional self-attention
reshaped = Reshape((-1, HIDDEN))(concat)
compatibilities = Dense(HIDDEN, activation='elu', use_bias=False, name='compatibilites')(reshaped)
attention_weights = Dense(HIDDEN, activation='softmax', use_bias=False, name='self_att_weights')(compatibilities)
attention_ = multiply([reshaped, attention_weights], name='attention_mul')
attention = Lambda(lambda x: K.sum(x, axis=1), name="attention")(attention_)
#attention = dot([reshaped, attention_weights], axes=-1, name='attention')

hidden_ = Dense(HIDDEN, activation='tanh', name='final_dense_1')(attention)
#hidden_2 = Dense(HIDDEN, activation='elu', name='final_dense_2')(hidden_1)
#hidden_ = Dropout(rate=0.25, name="hidden_dropout")(hidden_2)
#hidden = BatchNormalization()(hidden_)

preds = Dense(1, activation='sigmoid', name='predictions')(attention)#(hidden_)

CNN1 = Model(inputs=[inputs_q, inputs_a], outputs=[preds])
# try using different optimizers and different optimizer configs
CNN1.compile(loss='binary_crossentropy',
              optimizer='adam',#'adadelta',
              metrics=['accuracy']) # add mean_average_precision ?
print(CNN1.summary())

Build model...
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
question (InputLayer)           (None, 100)          0                                            
__________________________________________________________________________________________________
masking (Masking)               (None, 100)          0           question[0][0]                   
                                                                 answer[0][0]                     
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 300)     2394300     masking[0][0]                    
                                                                 masking[1][0]                    
______________________________________________________________________________________________

In [0]:
BATCH_SIZE = 32 # set to max allowable value
EPOCHS = 10*3

print('Train...')
CNN1.fit(train_X, train_y,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_data=(dev_X, dev_y),
          verbose=2,
          callbacks=None)

score, acc = CNN1.evaluate(test_X, test_y, batch_size=BATCH_SIZE)
print('Test score:', score)
print('Test accuracy:', acc)
# save model
CNN1.save('cnn_avgpool_final_B.h5')

Train...
Train on 2593 samples, validate on 495 samples
Epoch 1/30
 - 10s - loss: 0.6807 - acc: 0.5970 - val_loss: 0.6881 - val_acc: 0.5717
Epoch 2/30
 - 4s - loss: 0.6732 - acc: 0.5978 - val_loss: 0.6736 - val_acc: 0.5758
Epoch 3/30
 - 4s - loss: 0.6698 - acc: 0.5966 - val_loss: 0.7051 - val_acc: 0.5758
Epoch 4/30
 - 4s - loss: 0.6672 - acc: 0.5993 - val_loss: 0.6743 - val_acc: 0.5717
Epoch 5/30
 - 4s - loss: 0.6613 - acc: 0.6078 - val_loss: 0.6776 - val_acc: 0.5758
Epoch 6/30
 - 4s - loss: 0.6483 - acc: 0.6317 - val_loss: 0.6731 - val_acc: 0.5838
Epoch 7/30
 - 4s - loss: 0.6459 - acc: 0.6363 - val_loss: 0.7129 - val_acc: 0.5697
Epoch 8/30
 - 4s - loss: 0.6332 - acc: 0.6560 - val_loss: 0.7151 - val_acc: 0.5697
Epoch 9/30
 - 4s - loss: 0.6194 - acc: 0.6545 - val_loss: 0.6695 - val_acc: 0.5778
Epoch 10/30
 - 4s - loss: 0.6218 - acc: 0.6599 - val_loss: 0.7029 - val_acc: 0.5818
Epoch 11/30
 - 4s - loss: 0.6170 - acc: 0.6845 - val_loss: 0.7054 - val_acc: 0.5879
Epoch 12/30
 - 4s - loss: 0.

880/880 [==============================] - 0s 518us/step
Test score: 0.8541384447704662
Test accuracy: 0.6113636363636363


In [0]:
predict_and_save(CNN1, test_X, X["test"], "preds/cnn_avgpool_final_B.pred")

In [0]:
files.download('cnn_avgpool_final_B.h5')

##final BiGRU model

In [26]:
#
# BiGRU classifer
#
from keras.optimizers import *
sgd = SGD(lr=0.1, momentum=0.1, decay=0.99, nesterov=True)
#adagrad = Adagrad(lr=0.5, epsilon=None, decay=0.09)
activation = 'elu'
padding = 'valid'
filters = 300
kernel_size = 3

HIDDEN = 300

print('Build model...')
inputs_q = Input(shape=(MAX_SEQUENCE_LENGTH,), name='question')
inputs_a = Input(shape=(MAX_SEQUENCE_LENGTH,), name='answer')

# add masking
mask = Masking(mask_value=0., name='masking')
inputs_q_ = mask(inputs_q)
inputs_a_ = mask(inputs_a)

embed = Embedding(vocab_size+1, HIDDEN, weights=[embedding_matrix], trainable=False, mask_zero=False, name='embedding')#mask_zero=True,trainable=False
embed_q = embed(inputs_q_)
embed_a = embed(inputs_a_)

# biGRU one
gru_q1 = Bidirectional(myGRU(HIDDEN, return_sequences=True), merge_mode="concat")(embed_q)
gru_a1 = Bidirectional(myGRU(HIDDEN, return_sequences=True), merge_mode="concat")(embed_a)

pool1 = GlobalMaxPooling1D(name='pool1') #MAX
pool1_q = pool1(gru_q1)
pool1_a = pool1(gru_a1)

#outs = Dot(axes=-1,normalize=True)([pool1_q, pool1_a])

# biGRU two
gru_q2 = Bidirectional(myGRU(HIDDEN, return_sequences=True), merge_mode="concat")(gru_q1)
gru_a2 = Bidirectional(myGRU(HIDDEN, return_sequences=True), merge_mode="concat")(gru_a1)

pool2 = GlobalMaxPooling1D(name='pool2') #MAX
pool2_q = pool2(gru_q1)
pool2_a = pool2(gru_a1)

# concatenations
q_ = concatenate([pool1_q, pool2_q], name='question_concat')
a_ = concatenate([pool1_a, pool2_q], name='answer_concat')
re_q = Reshape((-1, HIDDEN))(q_)
re_a = Reshape((-1, HIDDEN))(a_)
q = Dropout(rate=0.25, name='question_dropout')(re_q)#(q_)
a = Dropout(rate=0.25, name='answer_dropout')(re_a)#(a_)

qa_mul = multiply([q, a], name='multiply') 
qa_sub = subtract([q, a], name='subtract')
qa_max = maximum([q, a], name='maximum')
qa_avg = average([q, a], name='average')
concat = concatenate([q, a, qa_max, qa_mul, qa_sub, qa_avg], name='final_concat')

# attention block - multi-dimensional self-attention
reshaped = Reshape((-1, HIDDEN))(concat)
compatibilities = Dense(HIDDEN, activation='elu', use_bias=False, name='compatibilites')(reshaped)
attention_weights = Dense(HIDDEN, activation='softmax', use_bias=False, name='self_att_weights')(compatibilities)
attention_ = multiply([reshaped, attention_weights], name='attention_mul')
attention = Lambda(lambda x: K.sum(x, axis=1), name="attention")(attention_)
#attention = dot([reshaped, attention_weights], axes=-1, name='attention')

hidden_ = Dense(HIDDEN, activation='tanh', name='final_dense_1')(attention)
#hidden_2 = Dense(HIDDEN, activation='elu', name='final_dense_2')(hidden_1)
#hidden_ = Dropout(rate=0.25, name="hidden_dropout")(hidden_2)
#hidden = BatchNormalization()(hidden_)

preds = Dense(1, activation='sigmoid', name='predictions')(attention)#(hidden_)

BiGRU = Model(inputs=[inputs_q, inputs_a], outputs=[preds])
# try using different optimizers and different optimizer configs
BiGRU.compile(loss='binary_crossentropy',
              optimizer='adam',#'adadelta',
              metrics=['accuracy']) # add mean_average_precision ?
print(BiGRU.summary())

Build model...
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
question (InputLayer)           (None, 100)          0                                            
__________________________________________________________________________________________________
answer (InputLayer)             (None, 100)          0                                            
__________________________________________________________________________________________________
masking (Masking)               (None, 100)          0           question[0][0]                   
                                                                 answer[0][0]                     
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 300)     2394300     masking[0][0]                

In [0]:
BATCH_SIZE = 32 # set to max allowable value
EPOCHS = 10*3

print('Train...')
BiGRU.fit(train_X, train_y,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_data=(dev_X, dev_y),
          verbose=2,
          callbacks=None)

score, acc = BiGRU.evaluate(test_X, test_y, batch_size=BATCH_SIZE)
print('Test score:', score)
print('Test accuracy:', acc)
# save model
BiGRU.save('bigru_maxpool_final_B.h5')

Train...
Train on 2593 samples, validate on 495 samples
Epoch 1/30
 - 17s - loss: 0.6714 - acc: 0.5943 - val_loss: 0.6778 - val_acc: 0.5717
Epoch 2/30
 - 11s - loss: 0.6362 - acc: 0.6155 - val_loss: 0.7571 - val_acc: 0.5717
Epoch 3/30
 - 11s - loss: 0.6057 - acc: 0.6834 - val_loss: 0.6661 - val_acc: 0.5980
Epoch 4/30
 - 11s - loss: 0.5626 - acc: 0.7181 - val_loss: 0.6601 - val_acc: 0.6040
Epoch 5/30
 - 11s - loss: 0.5259 - acc: 0.7493 - val_loss: 0.6951 - val_acc: 0.5596
Epoch 6/30
 - 11s - loss: 0.5138 - acc: 0.7698 - val_loss: 0.6663 - val_acc: 0.5939
Epoch 7/30
 - 11s - loss: 0.5015 - acc: 0.7682 - val_loss: 0.6854 - val_acc: 0.5919
Epoch 8/30
 - 11s - loss: 0.4807 - acc: 0.7802 - val_loss: 0.6923 - val_acc: 0.5657
Epoch 9/30
 - 11s - loss: 0.4680 - acc: 0.7813 - val_loss: 0.6833 - val_acc: 0.5758
Epoch 10/30
 - 11s - loss: 0.4375 - acc: 0.8118 - val_loss: 0.7031 - val_acc: 0.5758
Epoch 11/30
 - 11s - loss: 0.4150 - acc: 0.8122 - val_loss: 0.7076 - val_acc: 0.5616
Epoch 12/30
 - 11s

880/880 [==============================] - 1s 1ms/step
Test score: 0.8822599822824652
Test accuracy: 0.6


In [0]:
predict_and_save(BiGRU, test_X, X["test"], "preds/bigru_maxpool_final_B.pred")

In [0]:
files.download('bigru_maxpool_final_B.h5')

In [0]:
#
# BiGRU average pooling
#
from keras.optimizers import *
sgd = SGD(lr=0.1, momentum=0.1, decay=0.99, nesterov=True)
#adagrad = Adagrad(lr=0.5, epsilon=None, decay=0.09)
activation = 'elu'
padding = 'valid'
filters = 300
kernel_size = 3

HIDDEN = 300

print('Build model...')
inputs_q = Input(shape=(MAX_SEQUENCE_LENGTH,), name='question')
inputs_a = Input(shape=(MAX_SEQUENCE_LENGTH,), name='answer')

# add masking
mask = Masking(mask_value=0., name='masking')
inputs_q_ = mask(inputs_q)
inputs_a_ = mask(inputs_a)

embed = Embedding(vocab_size+1, HIDDEN, weights=[embedding_matrix], trainable=False, mask_zero=False, name='embedding')#mask_zero=True,trainable=False
embed_q = embed(inputs_q_)
embed_a = embed(inputs_a_)

# biGRU one
gru_q1 = Bidirectional(myGRU(HIDDEN, return_sequences=True), merge_mode="concat")(embed_q)
gru_a1 = Bidirectional(myGRU(HIDDEN, return_sequences=True), merge_mode="concat")(embed_a)

pool1 = GlobalAveragePooling1D(name='pool1')
pool1_q = pool1(gru_q1)
pool1_a = pool1(gru_a1)

# biGRU two
gru_q2 = Bidirectional(myGRU(HIDDEN, return_sequences=True), merge_mode="concat")(gru_q1)
gru_a2 = Bidirectional(myGRU(HIDDEN, return_sequences=True), merge_mode="concat")(gru_a1)

pool2 = GlobalAveragePooling1D(name='pool2')
pool2_q = pool2(gru_q1)
pool2_a = pool2(gru_a1)

# concatenations
q_ = concatenate([pool1_q, pool2_q], name='question_concat')
a_ = concatenate([pool1_a, pool2_q], name='answer_concat')
re_q = Reshape((-1, HIDDEN))(q_)
re_a = Reshape((-1, HIDDEN))(a_)
q = Dropout(rate=0.25, name='question_dropout')(re_q)#(q_)
a = Dropout(rate=0.25, name='answer_dropout')(re_a)#(a_)

qa_mul = multiply([q, a], name='multiply') 
qa_sub = subtract([q, a], name='subtract')
qa_max = maximum([q, a], name='maximum')
qa_avg = average([q, a], name='average')
concat = concatenate([q, a, qa_max, qa_mul, qa_sub, qa_avg], name='final_concat')

# attention block - multi-dimensional self-attention
reshaped = Reshape((-1, HIDDEN))(concat)
compatibilities = Dense(HIDDEN, activation='elu', use_bias=False, name='compatibilites')(reshaped)
attention_weights = Dense(HIDDEN, activation='softmax', use_bias=False, name='self_att_weights')(compatibilities)
attention_ = multiply([reshaped, attention_weights], name='attention_mul')
attention = Lambda(lambda x: K.sum(x, axis=1), name="attention")(attention_)
#attention = dot([reshaped, attention_weights], axes=-1, name='attention')

hidden_ = Dense(HIDDEN, activation='tanh', name='final_dense_1')(attention)
#hidden_2 = Dense(HIDDEN, activation='elu', name='final_dense_2')(hidden_1)
#hidden_ = Dropout(rate=0.25, name="hidden_dropout")(hidden_2)
#hidden = BatchNormalization()(hidden_)

preds = Dense(1, activation='sigmoid', name='predictions')(attention)#(hidden_)

BiGRU1 = Model(inputs=[inputs_q, inputs_a], outputs=[preds])
# try using different optimizers and different optimizer configs
BiGRU1.compile(loss='binary_crossentropy',
              optimizer='adam',#'adadelta',
              metrics=['accuracy']) # add mean_average_precision ?
print(BiGRU1.summary())

Build model...
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
question (InputLayer)           (None, 100)          0                                            
__________________________________________________________________________________________________
answer (InputLayer)             (None, 100)          0                                            
__________________________________________________________________________________________________
masking (Masking)               (None, 100)          0           question[0][0]                   
                                                                 answer[0][0]                     
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 300)     2394300     masking[0][0]                

In [0]:
BATCH_SIZE = 32 # set to max allowable value
EPOCHS = 10*2

print('Train...')
BiGRU1.fit(train_X, train_y,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_data=(dev_X, dev_y),
          verbose=2,
          callbacks=None)

score, acc = BiGRU1.evaluate(test_X, test_y, batch_size=BATCH_SIZE)
print('Test score:', score)
print('Test accuracy:', acc)
# save model
BiGRU1.save('bigru_avgpool_final_B.h5')

Train...
Train on 2593 samples, validate on 495 samples
Epoch 1/20
 - 18s - loss: 0.6782 - acc: 0.5974 - val_loss: 0.6848 - val_acc: 0.5717
Epoch 2/20
 - 11s - loss: 0.6710 - acc: 0.5978 - val_loss: 0.6765 - val_acc: 0.5717
Epoch 3/20
 - 11s - loss: 0.6519 - acc: 0.6140 - val_loss: 0.6927 - val_acc: 0.5798
Epoch 4/20
 - 11s - loss: 0.6353 - acc: 0.6309 - val_loss: 0.6730 - val_acc: 0.5697
Epoch 5/20
 - 11s - loss: 0.5922 - acc: 0.6799 - val_loss: 0.6790 - val_acc: 0.5737
Epoch 6/20
 - 11s - loss: 0.5509 - acc: 0.7246 - val_loss: 0.6882 - val_acc: 0.5818
Epoch 7/20
 - 11s - loss: 0.5261 - acc: 0.7428 - val_loss: 0.7173 - val_acc: 0.5697
Epoch 8/20
 - 11s - loss: 0.5110 - acc: 0.7563 - val_loss: 0.7265 - val_acc: 0.5475
Epoch 9/20
 - 11s - loss: 0.4928 - acc: 0.7728 - val_loss: 0.7369 - val_acc: 0.5616
Epoch 10/20
 - 11s - loss: 0.4756 - acc: 0.7728 - val_loss: 0.7409 - val_acc: 0.5717
Epoch 11/20
 - 11s - loss: 0.4602 - acc: 0.7813 - val_loss: 0.7474 - val_acc: 0.5818
Epoch 12/20
 - 11s

In [0]:
predict_and_save(BiGRU1, test_X, X["test"], "preds/bigru_avgpool_final_B.pred")

In [0]:
files.download('bigru_avgpool_final_B.h5')

In [0]:
print("finally done!")

finally done!


## Demo

In [0]:
import textwrap

def rank_rel_questions_for_question(model, data, proc_data, q_index):
    ind = q_index * 10
    example = data[ind : ind+10]
    print("QUESTION")
    print("\n".join(textwrap.wrap(example[0][0].body, 120, break_long_words=False)))
    print("\nRELATED QUESTIONS")
    for i in range(10):
        line = "{}) {}".format(i+1, example[i][1].body)
        print("\n".join(textwrap.wrap(line, 120, break_long_words=False)))
        print()
      
    proc_example_q = proc_data[0][ind:ind+10, :]
    proc_example_a = proc_data[1][ind:ind+10, :]
    proc_example = [proc_example_q, proc_example_a]
    
    probs, preds = predict(model, proc_example, 10)
    probs_preds = list(zip(probs, preds))
    ranked_probs_preds = sorted(enumerate(probs_preds), key=lambda x: -x[1][0])
    
    ranked_answers = []
    for rpp in ranked_probs_preds:
        idx = rpp[0]
        ranked_answers.append((example[idx][1].body, rpp[1][1]))
      
    print("\nRANKED RELATED QUESTIONS")
    for i in range(len(ranked_answers)):
        line = "{}) {}".format(i+1, ranked_answers[i][0])
        print("\n".join(textwrap.wrap(line, 120, break_long_words=False)))
        print("IS RELEVANT? {}".format(ranked_answers[i][1]))
        print()

In [44]:
rank_rel_questions_for_question(BiGRU, X["test"], test_X, 0)

QUESTION
Can someone suggest a link or explain how to obtain visas for family hoping to go on vacation to The UK and Spain? We
are bona fide residents in Qatar by the way. Thank you

RELATED QUESTIONS
1) Do you know how long it will take to get Schengen Visa from Embassy of Greece? 3 days? 1 week? any idea? Thanks.

2) Now that I have your attention (good morning btw ;)... Does anyone know if transit visa's are issued here?

3) Hi there. Can anyone tell me if there is a Swiss embassy in Doha ? If so , where is it ? Thanks

4) Ok - your nominations please for the "Most Useless Embassy" in Qatar? Who should have their little diplomatic flag
stuffed up their nostril? Who forgets who pays their salaries to swan around to cocktail parties? Who couldn't get care
less about the citizens as long as they get to have a shiny car? Who thinks a visa is a card to go shopping with? Please
Give me reasons.... no more than 300 words ... thanks a bundle.

5) Hi there, I am coming to doha on Thursday an

In [29]:
BATCH_SIZE = 32 # set to max allowable value
EPOCHS = 10

print('Train...')
BiGRU.fit(train_X, train_y,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_data=(dev_X, dev_y),
          verbose=2,
          callbacks=None)

score, acc = BiGRU.evaluate(test_X, test_y, batch_size=BATCH_SIZE)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Train on 2593 samples, validate on 495 samples
Epoch 1/10
 - 11s - loss: 0.5464 - acc: 0.7358 - val_loss: 0.6530 - val_acc: 0.5980
Epoch 2/10
 - 11s - loss: 0.5078 - acc: 0.7682 - val_loss: 0.6790 - val_acc: 0.6020
Epoch 3/10
 - 11s - loss: 0.4732 - acc: 0.7809 - val_loss: 0.6343 - val_acc: 0.6141
Epoch 4/10
 - 11s - loss: 0.4293 - acc: 0.8076 - val_loss: 0.6688 - val_acc: 0.6020
Epoch 5/10
 - 11s - loss: 0.3906 - acc: 0.8280 - val_loss: 0.6972 - val_acc: 0.6162
Epoch 6/10
 - 11s - loss: 0.3732 - acc: 0.8515 - val_loss: 0.7258 - val_acc: 0.6081
Epoch 7/10
 - 11s - loss: 0.3403 - acc: 0.8635 - val_loss: 0.7135 - val_acc: 0.6081
Epoch 8/10
 - 11s - loss: 0.3545 - acc: 0.8508 - val_loss: 0.7514 - val_acc: 0.5576
Epoch 9/10
 - 11s - loss: 0.4004 - acc: 0.8234 - val_loss: 0.7388 - val_acc: 0.6343
Epoch 10/10
 - 11s - loss: 0.3010 - acc: 0.8801 - val_loss: 0.7438 - val_acc: 0.6263
880/880 [==============================] - 1s 1ms/step
Test score: 0.7386161966757341
Test accuracy: 0.